In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,19577
2,Huelva,Confirmados PDIA 14 días,5335
3,Huelva,Tasa PDIA 14 días,"1039,616501354327"
4,Huelva,Confirmados PDIA 7 días,2525
5,Huelva,Total Confirmados,19800
6,Huelva,Curados,11044
7,Huelva,Fallecidos,176


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  19577.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  5243.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 245 personas en los últimos 7 días 

Un positivo PCR cada 119 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,19577.0,2525.0,5335.0,513170.0,492.039675,1039.616501,437.0
Huelva-Costa,11868.0,1567.0,3519.0,289548.0,541.188335,1215.342534,258.0
Condado-Campiña,5341.0,714.0,1260.0,156231.0,457.015573,806.498070,151.0
Huelva (capital),5243.0,587.0,1209.0,143837.0,408.100836,840.534772,96.0
Isla Cristina,1644.0,245.0,726.0,21393.0,1145.234422,3393.633431,38.0
Lepe,1562.0,200.0,409.0,27880.0,717.360115,1467.001435,32.0
Ayamonte,866.0,179.0,408.0,21104.0,848.180440,1933.282790,30.0
Palma del Condado (La),829.0,92.0,164.0,10801.0,851.772984,1518.377928,24.0
Punta Umbría,500.0,130.0,216.0,15355.0,846.629762,1406.707913,22.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Villalba del Alcor,385.0,73.0,170.0,3366.0,2168.746286,5050.505051,12.0
Berrocal,14.0,7.0,14.0,303.0,2310.231023,4620.462046,0.0
Encinasola,110.0,5.0,47.0,1310.0,381.679389,3587.786260,1.0
Isla Cristina,1644.0,245.0,726.0,21393.0,1145.234422,3393.633431,38.0
Paymogo,50.0,17.0,38.0,1162.0,1462.994836,3270.223752,4.0
Calañas,151.0,21.0,79.0,2768.0,758.670520,2854.046243,7.0
Villanueva de las Cruces,16.0,1.0,9.0,387.0,258.397933,2325.581395,0.0
San Bartolomé de la Torre,136.0,39.0,74.0,3761.0,1036.958256,1967.561819,3.0
Cerro de Andévalo (El),117.0,12.0,45.0,2327.0,515.685432,1933.820370,1.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Minas de Riotinto,95.0,2.0,19.0,3812.0,52.465897,498.426023,0.0,0.105263
Encinasola,110.0,5.0,47.0,1310.0,381.679389,3587.786260,1.0,0.106383
Villanueva de las Cruces,16.0,1.0,9.0,387.0,258.397933,2325.581395,0.0,0.111111
Aroche,80.0,1.0,5.0,3054.0,32.743942,163.719712,1.0,0.200000
Puebla de Guzmán,65.0,5.0,20.0,3092.0,161.707633,646.830530,1.0,0.250000
Cumbres Mayores,24.0,1.0,4.0,1749.0,57.175529,228.702115,0.0,0.250000
Calañas,151.0,21.0,79.0,2768.0,758.670520,2854.046243,7.0,0.265823
Cerro de Andévalo (El),117.0,12.0,45.0,2327.0,515.685432,1933.820370,1.0,0.266667
Almendro (El),32.0,2.0,7.0,840.0,238.095238,833.333333,0.0,0.285714
